In [1]:
#경고 끄기
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

plt.rc('font', family='Malgun Gothic')

In [3]:
def getData():
    df2008 = pd.read_csv("C:/농수산가격데이터/1001배추/2008010120081231.csv")
    df2009 = pd.read_csv("C:/농수산가격데이터/1001배추/2009010120091231.csv")
    df2010 = pd.read_csv("C:/농수산가격데이터/1001배추/2010010120101231.csv")
    df2011 = pd.read_csv("C:/농수산가격데이터/1001배추/2011010120111231.csv")
    df2012 = pd.read_csv("C:/농수산가격데이터/1001배추/2012010120121231.csv")
    df2013 = pd.read_csv("C:/농수산가격데이터/1001배추/2013010120131231.csv")
    df2014 = pd.read_csv("C:/농수산가격데이터/1001배추/2014010120141231.csv")
    df2015 = pd.read_csv("C:/농수산가격데이터/1001배추/2015010120151231.csv")
    df2016 = pd.read_csv("C:/농수산가격데이터/1001배추/2016010120161231.csv")
    df2017 = pd.read_csv("C:/농수산가격데이터/1001배추/2017010120171231.csv")
    df2018 = pd.read_csv("C:/농수산가격데이터/1001배추/2018010120181231.csv")
    return pd.concat((df2008, df2009, df2010, df2011, df2012, df2013, df2014, df2015, df2016, df2017, df2018), axis=0)

In [4]:
def priceTransform(df):
    #Kg당 가격(priceperkg)
    #총거래가격(totalprice) = 거래량(delngQy) * 가격(sbidpric)
    #총량(totalweight) = 거래단량(delngprut) * 거래량(delngQy) * 단위

    df11 = df[(df['stdunitnewcode']==11)]
    df12 = df[(df['stdunitnewcode']==12)]
    df13 = df[(df['stdunitnewcode']==13)]
    df11['priceperkg'] = df11['sbidpric'] / df11['delngprut'] * 1000
    df11['totalweight'] = df11['delngprut'] * df11['delngqy'] / 1000
    df12['priceperkg'] = df12['sbidpric'] / df12['delngprut']
    df12['totalweight'] = df12['delngprut'] * df12['delngqy']
    df13['priceperkg'] = df13['sbidpric'] / df13['delngprut'] / 1000
    df13['totalweight'] = df13['delngprut'] * df13['delngqy'] * 1000
    
    df = pd.concat([df11, df12, df13])

    df['totalprice'] = df['sbidpric'] * df['delngqy']
        
    return df

In [5]:
df = getData()

In [6]:
df = pricTransform(df)

In [ ]:
# 데이터 가공
# 2만 이상은 결측치라 판단 제거
df = df[(df['priceperkg'] <= 20000)]
# 가격이 100이하 제거
df = df[(df['priceperkg'] >= 100)]

# 근거자료
# https://www.kamis.or.kr/customer/main/main.do

# 무게가 0인 것 제거
df = df[(df['totalweight'] >= 0)]

# 결측값 공백으로 채우기
df['stdmtcnewnm'] = df['stdmtcnewnm'].fillna('')
# 데이터 도별로 구별
df['stdmtcnewnm'] = df['stdmtcnewnm'].map(lambda x: x.split(' ')[0])

df['yyyy'] = df['delngde'].map(lambda x: str(x)[0:4])
df['mm'] = df['delngde'].map(lambda x: str(x)[4:6])
df['dd'] = df['delngde'].map(lambda x: str(x)[6:8])

In [ ]:
df.to_csv('mean_of_month_1.csv')

In [ ]:
print(df[df['aucsenm'] == '경매'].shape[0]/df['aucsenm'].size)
print(df[df['aucsenm'] == '정가수의'].shape[0]/df['aucsenm'].size)
print(df[df['aucsenm'] == '자기계산'].shape[0]/df['aucsenm'].size)
print(df[df['aucsenm'] == '매수도매'].shape[0]/df['aucsenm'].size)

In [ ]:
f, ax = plt.subplots(figsize=(15, 4))
sns.countplot(df['aucsenm'])

In [ ]:
f, ax = plt.subplots(figsize=(20, 4))
df[['aucsenm','priceperkg']].groupby("aucsenm").mean().sort_values(by='priceperkg', ascending=False).plot.bar(ax=ax)

In [ ]:
pd.DataFrame(df[['aucsenm','totalprice']].groupby("aucsenm").sum().sort_values(by='totalprice', ascending=False)['totalprice'].apply(lambda x: format(int(x), ',')))

In [ ]:
f, ax = plt.subplots(figsize=(20, 4))
df[['aucsenm','totalprice']].groupby("aucsenm").sum().sort_values(by='totalprice', ascending=False).plot.bar(ax=ax)

In [ ]:
pd.DataFrame(df[['aucsenm','totalweight']].groupby("aucsenm").sum().sort_values(by='totalweight', ascending=False)['totalweight'].apply(lambda x: format(int(x), ',')))


In [ ]:
f, ax = plt.subplots(figsize=(20, 4))
df[['aucsenm','totalweight']].groupby("aucsenm").sum().sort_values(by='totalweight', ascending=False).plot.bar(ax=ax)

### 경락일자 delngde
### 거래단량 delngprut
### 거래량 delngqy

### 등급코드 stdqlitynewcode / stdqlitynewnm

In [ ]:
f, ax = plt.subplots(figsize=(15, 4))
sns.countplot(df['stdqlitynewnm'],order = df['stdqlitynewnm'].value_counts().index)

#### 그룹별 중위수 및 평균

In [ ]:
pd.concat([
    df[(df['stdqlitynewnm'] == '특') | (df['stdqlitynewnm'] == '무등급') | (df['stdqlitynewnm'] == '상') | (df['stdqlitynewnm'] == '보통')][['stdqlitynewnm','priceperkg']].groupby("stdqlitynewnm").median()
    , df[(df['stdqlitynewnm'] == '특') | (df['stdqlitynewnm'] == '무등급') | (df['stdqlitynewnm'] == '상') | (df['stdqlitynewnm'] == '보통')][['stdqlitynewnm','totalweight']].groupby("stdqlitynewnm").median()
    , df[(df['stdqlitynewnm'] == '특') | (df['stdqlitynewnm'] == '무등급') | (df['stdqlitynewnm'] == '상') | (df['stdqlitynewnm'] == '보통')][['stdqlitynewnm','priceperkg']].groupby("stdqlitynewnm").mean()
    , df[(df['stdqlitynewnm'] == '특') | (df['stdqlitynewnm'] == '무등급') | (df['stdqlitynewnm'] == '상') | (df['stdqlitynewnm'] == '보통')][['stdqlitynewnm','totalweight']].groupby("stdqlitynewnm").mean()
] , axis=1)

In [ ]:
f, ax = plt.subplots(figsize=(15, 4))
fig = sns.boxplot(x='stdqlitynewnm', y='priceperkg', data=df[(df['stdqlitynewnm'] == '특') | (df['stdqlitynewnm'] == '무등급') | (df['stdqlitynewnm'] == '상') | (df['stdqlitynewnm'] == '보통')])

In [ ]:
f, ax = plt.subplots(figsize=(15, 4))
fig = sns.boxplot(x='stdqlitynewnm', y='totalweight', data=df[(df['stdqlitynewnm'] == '특') | (df['stdqlitynewnm'] == '무등급') | (df['stdqlitynewnm'] == '상') | (df['stdqlitynewnm'] == '보통')])

In [ ]:
df[((df['stdqlitynewnm'] == '특') | (df['stdqlitynewnm'] == '무등급') | (df['stdqlitynewnm'] == '상') | (df['stdqlitynewnm'] == '보통')) & (df['totalweight'] <= 0)].shape

In [ ]:
# 10ton 밑으로 재조회
f, ax = plt.subplots(figsize=(15, 4))
fig = sns.boxplot(x='stdqlitynewnm', y='totalweight', data=df[((df['stdqlitynewnm'] == '특') | (df['stdqlitynewnm'] == '무등급') | (df['stdqlitynewnm'] == '상') | (df['stdqlitynewnm'] == '보통')) & (df['totalweight'] <= 10000)])
print(df[((df['stdqlitynewnm'] == '특') | (df['stdqlitynewnm'] == '무등급') | (df['stdqlitynewnm'] == '상') | (df['stdqlitynewnm'] == '보통')) & (df['totalweight'] >= 10000)].shape)

#### 그룹별 중위수

In [ ]:
df[(df['stdqlitynewnm'] != '특') & (df['stdqlitynewnm'] != '무등급') & (df['stdqlitynewnm'] != '상') & (df['stdqlitynewnm'] != '보통')][['stdqlitynewnm','priceperkg']].groupby("stdqlitynewnm").median()

#### 그룹별 평균

In [ ]:
df[(df['stdqlitynewnm'] != '특') & (df['stdqlitynewnm'] != '무등급') & (df['stdqlitynewnm'] != '상') & (df['stdqlitynewnm'] != '보통')][['stdqlitynewnm','priceperkg']].groupby("stdqlitynewnm").mean()

In [ ]:
f, ax = plt.subplots(figsize=(15, 4))
fig = sns.boxplot(x='stdqlitynewnm', y='priceperkg', data=df[(df['stdqlitynewnm'] != '특') & (df['stdqlitynewnm'] != '무등급') & (df['stdqlitynewnm'] != '상') & (df['stdqlitynewnm'] != '보통')])

### 품종코드 stdspciesnewcode / stdspciesnewnm
- 구품종코드 stdspciescode stdspciesnm
- 품목코드 stdprdlstnewcode / stdprdlstnewnm
- 구품목코드 stdprdlstnm / stdprdlstcode 

In [ ]:
df[['stdspciesnewcode','stdspciesnewnm']].drop_duplicates()

In [ ]:
f, ax = plt.subplots(figsize=(15, 4))
sns.countplot(df['stdspciesnewnm'],order = df['stdspciesnewnm'].value_counts().index)

In [ ]:
f, ax = plt.subplots(figsize=(15, 4))
fig = sns.boxplot(x='stdspciesnewnm', y='priceperkg', data=df)

In [ ]:
f, ax = plt.subplots(figsize=(15, 4))
fig = sns.boxplot(x='stdspciesnewnm', y='totalweight', data=df)

In [ ]:
f, ax = plt.subplots(figsize=(15, 4))
fig = sns.boxplot(x='stdspciesnewnm', y='totalweight', data=df[(df['totalweight'] <= 10000)])

#### 년도별 분포 시각화
- 참고자료 : 1_1. 배추무게분포.ipynb

### 산지코드 stdmtcnewcode / stdmtcnewnm
- 구산지코드 cprmtccode / cprmtcnm

In [ ]:
# 결측값이 너무 많으므로 처리방안 고려 필요
df[df['stdmtcnewnm'] == ''].shape

In [ ]:
# df[['stdmtcnewcode','stdmtcnewnm']].drop_duplicates()
df[['stdmtcnewnm']].drop_duplicates()

In [ ]:
f, ax = plt.subplots(figsize=(20, 4))
df[['stdmtcnewnm','totalweight']].groupby("stdmtcnewnm").sum().sort_values(by='totalweight', ascending=False).plot.bar(ax=ax)

In [ ]:
f, ax = plt.subplots(figsize=(20, 4))
df[['stdmtcnewnm','totalprice']].groupby("stdmtcnewnm").sum().sort_values(by='totalprice', ascending=False).plot.bar(ax=ax)

### 단위코드 stdunitnewcode / stdunitnewnm

In [ ]:
df[['stdunitnewcode','stdunitnewnm']].drop_duplicates()

In [ ]:
sns.countplot(df['stdunitnewnm'])

In [ ]:
sns.countplot(df[(df['stdunitnewcode'] != 12)]['stdunitnewnm'])

### 시장코드 whsalmrktnewcode / whsalmrktnewnm
- 구시장코드 whsalmrktcode / whsalmrktnm 
- 가락시장 코드 1005601(신) 110001(구)

In [ ]:
f, ax = plt.subplots(figsize=(20, 4))
df[['whsalmrktnewnm','totalweight']].groupby("whsalmrktnewnm").sum().sort_values(by='totalweight', ascending=False).plot.bar(ax=ax)

In [ ]:
f, ax = plt.subplots(figsize=(20, 4))
df[['whsalmrktnewnm','totalprice']].groupby("whsalmrktnewnm").sum().sort_values(by='totalprice', ascending=False).plot.bar(ax=ax)

In [ ]:
f, ax = plt.subplots(figsize=(10, 4))
sns.distplot(df['priceperkg'])

In [ ]:
# 로그스케일링 histogram
f, ax = plt.subplots(figsize=(10, 4))
sns.distplot(np.log1p(df['priceperkg']))

In [ ]:
new_df = pd.concat((df[['priceperkg','yyyy','mm']].groupby(['yyyy','mm']).mean(), df.groupby(['yyyy','mm'])[['totalprice','totalweight']].sum()), axis=1)

In [ ]:
import datetime as dt
from dateutil.relativedelta import relativedelta
yyyymm_1 = '200801'
yyyymm_2 = '201812'

MONTH = relativedelta(months=+1)

fmt = '%Y%m'
date_1 = dt.datetime.strptime(yyyymm_1, fmt).date()
date_2 = dt.datetime.strptime(yyyymm_2, fmt).date()

yyyymm = []
while date_1 <= date_2:
    yyyymm.append(date_1.strftime('%Y')+date_1.strftime('%m'))
    date_1 += MONTH

In [ ]:
new_df = pd.DataFrame(new_df.values)

In [ ]:
new_df['yyyy'] = new_df['date'].map(lambda x: str(x)[0:4])
new_df['mm'] = new_df['date'].map(lambda x: str(x)[4:6])

In [ ]:
new_df[new_df['yyyy']=='2008']

In [ ]:
f, ax = plt.subplots(15,0)
ax = sns.lineplot(x="mm", y=0 , data=new_df[new_df['yyyy']=='2008'])

In [ ]:
yyyy = sorted(new_df['yyyy'].unique())

In [ ]:
type(yyyy[0])

In [ ]:
new_df[new_df['yyyy']=='2018']

In [ ]:
yyyy = sorted(new_df['yyyy'].unique())
f, ax = plt.subplots(20,0)
for i, v in enumerate(yyyy):
    ax = sns.lineplot(x="mm", y=0, data=new_df[new_df['yyyy']==v])

In [ ]:
new_df = pd.concat((df[['priceperkg','yyyy','mm']].groupby(['yyyy','mm']).mean(), df.groupby(['yyyy','mm'])[['totalprice','totalweight']].sum()), axis=1)

In [ ]:
mean_of_month_1 = 